In [1]:
import os
import random
import logging
import numpy as np
from tensorboardX import SummaryWriter

import data_p
import model
import utils

import torch
import torch.optim as optim
import torch.nn.functional as F

%load_ext tensorboard
%load_ext tensorboard.notebook
%tensorboard --logdir logs

SAVES_DIR = "saves"

BATCH_SIZE = 16
LEARNING_RATE = 1e-4
MAX_EPOCHS = 10000

log = logging.getLogger("train")



def run_test(test_data, net, end_token, device='cpu'):
    """Runs every epoch to calculate the BLEU score on the test dataset. Test data is now group by the first phrase, with a shape
    (first_phrase, [second_phrases]). BLEU score now calculated by a function accepting several reference sequences & returns the best score for them."""
    bleu_sum = 0.0
    bleu_count = 0
    for p1, p2 in test_data:
        input_seq = model.pack_input(p1, net.emb, device)
        enc = net.encode(input_seq)
        _, tokens = net.decode_chain_argmax(enc, input_seq.data[0:1], seq_len=data_p.MAX_TOKENS, stop_at_token=end_token)
        ref_indices = [indices[1:] for indices in p2]
        bleu_sum += utils.calc_bleu_many(tokens, ref_indices)
        bleu_count += 1
    return bleu_sum / bleu_count



if __name__ == "__main__":
    logging.basicConfig(format="%(asctime)-15s %(levelname)s %(message)s", level=logging.INFO)
  
    genre = "comedy"
    name = "inigo"
    load = "/workspace/epoch_040_0.579_0.113_.dat"
    samples = 4
    disable_skip = False
    device = "cuda"

    saves_path = os.path.join(SAVES_DIR, name)
    os.makedirs(saves_path, exist_ok=True)
    #loads the training data
    phrase_pairs, emb_dict = data_p.load_data(genre_filter=genre)
    log.info("Obtained %d phrase pairs with %d uniq words", len(phrase_pairs), len(emb_dict))
    data_p.save_emb_dict(saves_path, emb_dict)
    end_token = emb_dict[data_p.END_TOKEN]
    train_data = data_p.encode_phrase_pairs(phrase_pairs, emb_dict)
    rand = np.random.RandomState(data_p.SHUFFLE_SEED)
    rand.shuffle(train_data)
    train_data, test_data = data_p.split_train_test(train_data)
    log.info("Training data converted, got %d samples", len(train_data))
    train_data = data_p.group_train_data(train_data) #groups the training data by the first phrase
    test_data = data_p.group_train_data(test_data) #groups the test data by the first phrase
    log.info("Train set has %d phrases, test %d", len(train_data), len(test_data))
    #creates model and loads its weights from the given file (from the previous cross-entropy training in our case)
    rev_emb_dict = {idx: word for word, idx in emb_dict.items()}

    net = model.PhraseModel(emb_size=model.EMBEDDING_DIM, dict_size=len(emb_dict), hid_size=model.HIDDEN_STATE_SIZE).to(device)
    log.info("Model: %s", net)

    writer = SummaryWriter(comment="-" + name)
    net.load_state_dict(torch.load(load))
    log.info("Model loaded from %s, continue training in RL mode...")
    #create tensor with #BEG token that will be used to look up the embeddings and pass the result to the decoder
    beg_token = torch.LongTensor([emb_dict[data_p.BEGIN_TOKEN]]).to(device)

    with utils.Tracker(writer, batch_size=100) as tracker:
        optimiser = optim.Adam(net.parameters(), lr=LEARNING_RATE, eps=1e-3)
        batch_idx = 0
        best_bleu = None
        for epoch in range(MAX_EPOCHS):
            random.shuffle(train_data)
            dial_shown = False
            total_samples = 0
            skipped_samples = 0
            bleus_argmax = []
            bleus_sample = []
            #pack the batch, encode all first sequences, then declare lists that will be populated during individual decoding of batch entries
            for batch in data_p.iterate_batches(train_data, BATCH_SIZE):
                batch_idx += 1
                optimiser.zero_grad()
                input_seq, input_batch, output_batch = model.pack_batch_no_out(batch, net.emb, device)
                enc = net.encode(input_seq)

                net_policies = []
                net_actions = []
                net_advantages = []
                beg_embeddings = net.emb(beg_token)
                #process individual entries: first strip the #BEG token from reference seqs and obtain individual entry for encoded batch
                for idx, inp_idx in enumerate(input_batch):
                    total_samples += 1
                    ref_indices = [indices[1:] for indices in output_batch[idx]]
                    item_enc = net.get_encoded_item(enc, idx)
                    #then decode the batch entry in argmax mode and calculate its BLEU score, which will be used as a baseline in REINFORCE
                    r_argmax, actions = net.decode_chain_argmax(item_enc, beg_embeddings, data_p.MAX_TOKENS, stop_at_token=end_token)
                    argmax_bleu = utils.calc_bleu_many(actions, ref_indices)
                    bleus_argmax.append(argmax_bleu)

                    if not disable_skip and argmax_bleu > 0.99:
                        skipped_samples += 1
                        continue
                    #the following lines are executed once every epoch and only for the sake of getting some information during training
                    if not dial_shown:
                        log.info("Input: %s", utils.untokenize(data_p.decode_words(inp_idx, rev_emb_dict)))
                        ref_words = [utils.untokenize(data_p.decode_words(ref, rev_emb_dict)) for ref in ref_indices]
                        log.info("Refer: %s", " ~~|~~ ".join(ref_words))
                        log.info("Argmax: %s, bleu=%.4f", utils.untokenize(data_p.decode_words(actions, rev_emb_dict)), argmax_bleu)
          
                    for _ in range(samples):
                        r_sample, actions = net.decode_chain_sampling(item_enc, beg_embeddings, data_p.MAX_TOKENS, stop_at_token=end_token, device=device)
                        sample_bleu = utils.calc_bleu_many(actions, ref_indices)

                        if not dial_shown:
                            log.info("Sample: %s, bleu=%.4f", utils.untokenize(data_p.decode_words(actions, rev_emb_dict)), sample_bleu)
              
                        net_policies.append(r_sample)
                        net_actions.extend(actions)
                        net_advantages.extend([sample_bleu - argmax_bleu] * len(actions))
                        bleus_sample.append(sample_bleu)
                    dial_shown = True
        
                if not net_policies:
                    continue
        
                policies_v = torch.cat(net_policies)
                actions_t = torch.LongTensor(net_actions).to(device)
                adv_v = torch.FloatTensor(net_advantages).to(device)

                log_prob_v = F.log_softmax(policies_v, dim=1)
                log_prob_actions_v = adv_v * log_prob_v[range(len(net_actions)), actions_t]
                loss_policy_v = -log_prob_actions_v.mean()

                loss_v = loss_policy_v
                loss_v.backward()
                optimiser.step()

                tracker.track("advantage", adv_v, batch_idx)
                tracker.track("loss_policy", loss_policy_v, batch_idx)

            bleu_test = run_test(test_data, net, end_token, device)
            bleu = np.mean(bleus_argmax)
            writer.add_scalar("bleu_test", bleu_test, batch_idx)
            writer.add_scalar("bleu_argmax", bleu, batch_idx)
            writer.add_scalar("bleu_sample", np.mean(bleus_sample), batch_idx)
            writer.add_scalar("skipped_samples", skipped_samples / total_samples, batch_idx)
            writer.add_scalar("epoch", batch_idx, epoch)
            log.info("Epoch %d, test BLEU: %.3f", epoch, bleu_test)

            if best_bleu is None or best_bleu < bleu_test:
                best_bleu = bleu_test
                log.info("Best bleu updated: %.4f", bleu_test)
                torch.save(net.state_dict(), os.path.join(saves_path, "bleu_%.3f_%02d.dat" % (bleu_test, epoch)))
            if epoch % 10 == 0:
                torch.save(net.state_dict(), os.path.join(saves_path, "epoch_%03d_%.3f_%.3f.dat" % (epoch, bleu, bleu_test)))
    writer.close()

The tensorboard module is not an IPython extension.


Reusing TensorBoard on port 6006 (pid 409), started 3 days, 5:41:21 ago. (Use '!kill 409' to kill it.)

2019-12-15 20:16:09,494 INFO Loaded 159 movies with genre comedy
2019-12-15 20:16:09,495 INFO Read and tokenise phrases...
2019-12-15 20:16:21,596 INFO Loaded 93039 phrases
2019-12-15 20:16:22,104 INFO Loaded 24716 dialogues with 93039 phrases, generating training pairs
2019-12-15 20:16:22,261 INFO Counting freq of words...
2019-12-15 20:16:22,910 INFO Data has 31774 uniq words, 4913 of them occur more than 10
2019-12-15 20:16:23,334 INFO Obtained 47644 phrase pairs with 4905 uniq words
2019-12-15 20:16:24,037 INFO Training data converted, got 25166 samples
2019-12-15 20:16:24,115 INFO Train set has 21672 phrases, test 1253
2019-12-15 20:16:30,989 INFO Model: PhraseModel(
  (emb): Embedding(4905, 50)
  (encoder): LSTM(50, 512, batch_first=True)
  (decoder): LSTM(50, 512, batch_first=True)
  (output): Sequential(
    (0): Linear(in_features=512, out_features=4905, bias=True)
  )
)
2019-12-15 20:16:31,037 INFO Model loaded from %s, continue training in RL mode...
2019-12-15 20:16:31,116 

2019-12-15 22:54:09,292 INFO Argmax: no, right. #END, bleu=0.2596
2019-12-15 22:54:09,298 INFO Sample: i understand understand. #END, bleu=0.0670
2019-12-15 22:54:09,304 INFO Sample: no, man. #END, bleu=0.2596
2019-12-15 22:54:09,313 INFO Sample: no, well, don't worry. #END, bleu=0.2315
2019-12-15 22:54:09,320 INFO Sample: no, we're. #END, bleu=0.2596
2019-12-15 23:07:52,152 INFO Epoch 11, test BLEU: 0.139
2019-12-15 23:07:52,158 INFO Best bleu updated: 0.1394
2019-12-15 23:07:52,230 INFO Input: #BEG i think i love you, mate. how do you know so much about so much? #END
2019-12-15 23:07:52,231 INFO Refer: im a hooker. #END
2019-12-15 23:07:52,232 INFO Argmax: my leg. #END, bleu=0.3179
2019-12-15 23:07:52,239 INFO Sample: just johnny. #END, bleu=0.3179
2019-12-15 23:07:52,246 INFO Sample: i'm fine that's the. #END, bleu=0.2582
2019-12-15 23:07:52,252 INFO Sample: a sure voice. #END, bleu=0.3873
2019-12-15 23:07:52,260 INFO Sample: i'm with my.. #END, bleu=0.2582
2019-12-15 23:21:37,009 I

2019-12-16 02:00:52,217 INFO Epoch 24, test BLEU: 0.141
2019-12-16 02:00:52,220 INFO Best bleu updated: 0.1410
2019-12-16 02:00:52,301 INFO Input: #BEG they'd never talk in front of me. they'd walk away up the beach. nervous about something. #END
2019-12-16 02:00:52,302 INFO Refer: who was? #END
2019-12-16 02:00:52,302 INFO Argmax: all right, all. #END, bleu=0.0577
2019-12-16 02:00:52,308 INFO Sample: one right. #END, bleu=0.0913
2019-12-16 02:00:52,313 INFO Sample: i'm months. #END, bleu=0.0913
2019-12-16 02:00:52,317 INFO Sample: oh. #END, bleu=0.0925
2019-12-16 02:00:52,324 INFO Sample: they're didn't what. #END, bleu=0.0707
2019-12-16 02:14:36,970 INFO Epoch 25, test BLEU: 0.141
2019-12-16 02:14:36,973 INFO Best bleu updated: 0.1413
2019-12-16 02:14:37,037 INFO Input: #BEG he's gonna hit ya with 20 grand, eddie. how much cash we got? #END
2019-12-16 02:14:37,038 INFO Refer: not enough to cover a bet that big. #END
2019-12-16 02:14:37,039 INFO Argmax: not enough to cover a bet that.

...

2019-12-17 10:13:27,825 INFO Sample: oh, yeah. #END, bleu=0.1425
2019-12-17 10:13:27,831 INFO Sample: oh, yeah. #END, bleu=0.1425
2019-12-17 10:13:27,838 INFO Sample: yeah,... #END, bleu=0.1624
2019-12-17 10:28:22,067 INFO Epoch 162, test BLEU: 0.142
2019-12-17 10:28:22,108 INFO Input: #BEG he's like the pope. like we're gonna snake a girl away from the pope. #END
2019-12-17 10:28:22,109 INFO Refer: hey, let's go spy on them. #END
2019-12-17 10:28:22,110 INFO Argmax: i can't believe that you is. #END, bleu=0.1668
2019-12-17 10:28:22,118 INFO Sample: i can't do it. #END, bleu=0.1566
2019-12-17 10:28:22,128 INFO Sample: i'm telling you, i. you. #END, bleu=0.2041
2019-12-17 10:28:22,138 INFO Sample: you can you, can't you. #END, bleu=0.2043
2019-12-17 10:28:22,147 INFO Sample: i can't believe you,. #END, bleu=0.2008
2019-12-17 10:42:21,458 INFO Epoch 163, test BLEU: 0.141
2019-12-17 10:42:21,498 INFO Input: #BEG i just wish he never brought it up. ricky won't shut up about it. #END
2019-1

2019-12-17 13:37:02,649 INFO Sample: i did you think. #END, bleu=0.1918
2019-12-17 13:37:02,656 INFO Sample: i was you. #END, bleu=0.1740
2019-12-17 13:37:02,666 INFO Sample: i haven't you it. you. #END, bleu=0.2043
2019-12-17 13:37:02,673 INFO Sample: i don't you you. #END, bleu=0.1918
2019-12-17 13:51:47,708 INFO Epoch 176, test BLEU: 0.141
2019-12-17 13:51:47,748 INFO Input: #BEG i just called to say that i'm having a really great time. #END
2019-12-17 13:51:47,749 INFO Refer: good! you should go over and watch some ted carson. #END
2019-12-17 13:51:47,749 INFO Argmax: i, you. you. #END, bleu=0.1134
2019-12-17 13:51:47,758 INFO Sample: so you, actually, you. #END, bleu=0.1239
2019-12-17 13:51:47,765 INFO Sample: i, you. you. #END, bleu=0.1134
2019-12-17 13:51:47,771 INFO Sample: well, you think so. #END, bleu=0.1134
2019-12-17 13:51:47,778 INFO Sample: i, you. you. #END, bleu=0.1134
2019-12-17 14:06:08,208 INFO Epoch 177, test BLEU: 0.141
2019-12-17 14:06:08,258 INFO Input: #BEG chr

2019-12-17 17:01:02,718 INFO Input: #BEG oh, uh, we're gonna go in the studio and we're, we're doin' a new album. #END
2019-12-17 17:01:02,719 INFO Refer: yeah? #END
2019-12-17 17:01:02,720 INFO Argmax: what? #END, bleu=0.5774
2019-12-17 17:01:02,728 INFO Sample: what? #END, bleu=0.5774
2019-12-17 17:01:02,734 INFO Sample: what? #END, bleu=0.5774
2019-12-17 17:01:02,741 INFO Sample: what? #END, bleu=0.5774
2019-12-17 17:01:02,745 INFO Sample: what? #END, bleu=0.5774
2019-12-17 17:15:13,741 INFO Epoch 190, test BLEU: 0.140
2019-12-17 17:15:13,803 INFO Input: #BEG if you promise not to snap my head off, i'd like to thank you. #END
2019-12-17 17:15:13,804 INFO Refer: forget it. i didn't do it for you. his voice got on my nerves. #END
2019-12-17 17:15:13,805 INFO Argmax: yeah. i. you. #END, bleu=0.1360
2019-12-17 17:15:13,816 INFO Sample: yeah. i. you. #END, bleu=0.1360
2019-12-17 17:15:13,823 INFO Sample: no. i. you. #END, bleu=0.1360
2019-12-17 17:15:13,827 INFO Sample: no. #END, bleu=0.

2019-12-17 20:27:31,260 INFO Epoch 203, test BLEU: 0.138
2019-12-17 20:27:31,331 INFO Input: #BEG admiral- how am i going to handle all that weight? #END
2019-12-17 20:27:31,332 INFO Refer: you'll work it out, scotty. and remember: two of them. #END
2019-12-17 20:27:31,333 INFO Argmax: i don't know. you. #END, bleu=0.0852
2019-12-17 20:27:31,344 INFO Sample: i don't know. you. #END, bleu=0.0852
2019-12-17 20:27:31,354 INFO Sample: i don't know. you. #END, bleu=0.0852
2019-12-17 20:27:31,363 INFO Sample: i have to you. #END, bleu=0.0576
2019-12-17 20:27:31,372 INFO Sample: you're not. you. #END, bleu=0.0706
2019-12-17 20:42:40,900 INFO Epoch 204, test BLEU: 0.138
2019-12-17 20:42:40,947 INFO Input: #BEG oh. yes? just a minute. if you don't mind-- this is sort of personal #END
2019-12-17 20:42:40,949 INFO Refer: all right. have a nice christmas. #END
2019-12-17 20:42:40,949 INFO Argmax: that's it. it's you. i think. #END, bleu=0.1826
2019-12-17 20:42:40,959 INFO Sample: all right. thank 

FileNotFoundError: [Errno 2] No such file or directory: 'saves/inigo/epoch_210_0.341_0.139.dat'